<a href="https://colab.research.google.com/github/Arta-DS/DS/blob/main/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F_7_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [ ]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2020-06-25 16:05:44--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.227.11
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.227.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt’

nietzsche.txt       100%[===================>] 586.82K  1.22MB/s    in 0.5s    

2020-06-25 16:05:45 (1.22 MB/s) - ‘nietzsche.txt’ saved [600901/600901]



In [ ]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [ ]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [ ]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [ ]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [ ]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [ ]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]), tensor(23))

In [ ]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()

        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0])
        return predictions

In [ ]:
model = NeuralNetwork(nn.GRU, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [ ]:
model(X[0:1])

tensor([[ 0.2084, -0.1216, -0.0308, -0.2069, -0.1732,  0.0782, -0.0378, -0.0710,
          0.2652, -0.0983,  0.1712, -0.0029, -0.0701, -0.4059, -0.0797,  0.0451,
         -0.0206,  0.0052,  0.1072,  0.1700,  0.0527,  0.1813, -0.1920, -0.0605,
          0.0339,  0.2509, -0.1098]], grad_fn=<AddmmBackward>)

In [ ]:
model = model.cuda()

In [ ]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [ ]:
generate_text()

 and cicero swelling twice and sinking t|znpgkeamfsffxfzriyzysyttybk qo ljgssoycu


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()


Epoch 0. Time: 5.214, Train loss: 2.104
haps even the most remarkable creation o|nwtred arsing has pore their iss a foll 



Epoch 1. Time: 4.951, Train loss: 1.714
o longer either desired or fully develop|himent the smar sour evil to rumuact let



Epoch 2. Time: 5.267, Train loss: 1.580
mistrust of this species and of its seri|able of that eyceally a but foror fortru



Epoch 3. Time: 5.236, Train loss: 1.502
 today as can be readily disclosed and c|ultated too peop mething every and would



Epoch 4. Time: 5.102, Train loss: 1.449
 such things of pain and ill belong to t|he chlieve wead reasor a man chrisedly d


...


Epoch 95. Time: 5.455, Train loss: 1.028
the distrustful constraint in the commun|ity done of the conduction of refined at



Epoch 96. Time: 5.471, Train loss: 1.034
 creature is far too easily frightened h|imself the permans of an intercise human



Epoch 97. Time: 5.480, Train loss: 1.031
things and how many things which no long|er envicurres to be made calarance and w



Epoch 98. Time: 5.390, Train loss: 1.029
ion to his pupils knowledge for its own |no all misunderstand him for the reasona



Epoch 99. Time: 5.355, Train loss: 1.026
 as to overlook his own interests for he|re as a proudes the feeling of double mo
